
# Урок 6. Расчёт длительности А/B теста + продвинутые топики

In [37]:
#from math import asin

#import numpy as np

from scipy import stats
from statsmodels.stats.power import TTestIndPower, tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd

from typing import Union

#import plotly.graph_objects as go

In [2]:
#Расчёт effect_size для пропорций (качественная)
def calc_propotion_effect_size(conv1: int, nobs1: int, conv2: int, nobs2: int):
    p1, p2 = conv1 / nobs1, conv2 / nobs2
    
    # формула
    #return 2  * asin(np.sqrt(p1)) - 2 * asin(np.sqrt(p2))
    
    # библиотека
    return proportion_effectsize(p1, p2)

#Расчёт effect_size для непрерывных метрик (количественная)
def calc_continuous_effect_size(mean1: Union[float, int],
                                std1: Union[float, int],
                                mean2: Union[float, int],
                                std2: Union[float, int],
                                nobs1: int = 10_000,
                                nobs2: int = 10_000):
    
    # формула
    #return (mean1 - mean2) / np.sqrt((std1**2 + std2**2) / 2)
    
    # библиотека
    return effectsize_smd(mean1, std1, nobs1, mean2, std2, nobs2)[0]

In [3]:
#Расчёт минимально необходимой выборки для пропорций
def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n)

#Расчёт минимально необходимой выборки для непрерывной метрики
def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n)

## Задание 1

Мы хотим провести АБ-тест баннеров. Нынешняя конверсия в клик на баннер (CTR) равна 1.5%. Мы предполагаем, что с новыми баннерами CTR вырастет до 1.7%. Определите, сколько юзеров нам нужно отправить на каждую версию с уровнем доверия 95%.

In [24]:
p1, p2, gamma = 1.5/100, 1.7/100, 0.95
alpha = 1-gamma
beta = 0.2

In [25]:
effect_size = proportion_effectsize(p1, p2)

analysis = TTestIndPower()
size = analysis.solve_power(effect_size=effect_size, nobs1=None, alpha=alpha, power=1-beta, ratio=1, alternative='two-sided')
int(size) # объем одной выборки

61727

In [26]:
zt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=1 - beta)

61726.47090823281

## Задание 2

Мы хотим провести АБ-тест формы подтверждения заказа. Нынешняя конверсия в заказ равна 3%. Мы предполагаем, что конверсия вырастет на 0,1%. Определите, сколько юзеров нам нужно отправить на каждую версию с уровнем доверия 99% и уровнем мощности 90%.

In [27]:
p1, p2, gamma = 3/100, 3.1/100, 0.99
alpha = 1-gamma
beta = 0.1

In [28]:
effect_size = proportion_effectsize(p1, p2)

analysis = TTestIndPower()
size = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=1-beta, alternative='two-sided')
int(size)

879903

In [29]:
zt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=1 - beta)

879901.5736297085

## Задание 3

Дизайнер предложил добавить рекомендательную систему на этапе формирования корзины. Нынешний средний чек равен 2167 рублей, а стандартная ошибка (SD) равна 69. Мы предполагаем, что в новой версии средний чек вырастет 2180. Определите, сколько юзеров нам нужно отправить на каждую версию с уровнем доверия 95%.

https://www.statulator.com/SampleSize/ss2M.html

In [34]:
Mx, My = 2167, 2180
SDx, SDy = 69, 69
alpha = 0.05

effect_size = calc_continuous_effect_size(Mx, SDx, My, SDy)
effect_size

-0.18839873108913158

In [35]:
tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=0.8, ratio=1)

443.22546990429726

## Задание 4

Вы решили сравниваем метрику деньги на юзера в двух группах. Размер выборки - 1000 элементов в каждой группе. \
Для проверки нормальности распределения на выборке в 1000 наблюдений применили, критерий Шапиро-Уилка и получили p-value, равный 0.00002, при alpha = 5%, то какой бы вывод мы могли сделать в данном случае? \
В этом случае какой статистический критерий для проверки первоначальной гипотезы тут лучше всего подойдёт и почему?

Т.к. $p-valeu < \alpha$, то гипотезу $H_0$ о нормальности распределения отклоняем.

Количественые данные ->Распределение не нормальное -> 2 независимые выборки = критерий Манна-Уитни.

In [38]:
size = 10_000
normal_distr = stats.norm.rvs(size=size)
uniform_distr = stats.uniform.rvs(size=size)
expon_distr = stats.expon.rvs(size=size)

In [ ]:
#pip install distfit
from distfit import distfit

# Initialize for discrete distribution fitting
dfit = distfit(distr=[
    'norm', 'uniform', 'expon'
])

# Run distfit to and determine whether we can find the parameters from the data.
result = dfit.fit_transform(uniform_distr)

In [ ]:
result['model']